In [2]:
import bsscraper
import importlib
importlib.reload(bsscraper)

<module 'bsscraper' from '/Users/stevennewton/Documents/GitHub/MLB_Statcast/notebooka/bsscraper.py'>

In [ ]:
# Database for two years and two teams
bsscraper.make_database('NYY_NYM_19_20.db', [2019,2020], ['NYY,NYM'])

In [3]:
# Csv for 2020 Yankees only.
bsscraper.savant_call(2020, "NYY", csv=True)

('NYY_2020.csv', <http.client.HTTPMessage at 0x7fc722c342b0>)

In [6]:
import sqlite3
import pandas as pd

# Connect to, query, and load database into dataframe.
conn = sqlite3.connect('NYY_NYM_19_20.db')
df = pd.read_sql_query('SELECT player_name, vx0 FROM statcast;', conn)
conn.close()
df

,player_name,vx0
0,Aroldis Chapman,1.657994
1,Aroldis Chapman,-0.195399
2,Aroldis Chapman,1.476399
3,Aroldis Chapman,1.765126
4,Aroldis Chapman,-0.735986
...,...,...
69003,Jacob deGrom,8.090785
69004,Jacob deGrom,3.951370
69005,Jacob deGrom,8.955589
69006,Jacob deGrom,7.089980


In [8]:
#To verify this has every pitch, I'll estimate how many rows I expect for 2 teams over 2 years
est_pitches_per_game=150
games=162+60   # shortened 2020 season
f'approximately {est_pitches_per_game*games*2} pitches'

'approximately 66600 pitches'